In [ ]:
from pandas_datareader import data as pdr
from yahoo_fin import stock_info as si
import yfinance as yf
import pandas as pd
import datetime
import time
import talib
from talib import *
import matplotlib.pyplot as plt
import math
import numpy as np
import os
from LSSVMRegression import LSSVMRegression
os.environ["PATH"] += os.pathsep + r'C:\Program Files\Graphviz\bin'
yf.pdr_override()

In [ ]:
def add_indicators():
        
    R_S_I = RSI(df['Adj Close'], timeperiod=slow)
    E_M_A = EMA(df['Adj Close'], timeperiod=fast)
    macd, macdsig, macdhist = MACD(df['Adj Close'], fastperiod=fast, slowperiod=slow, signalperiod=really_fast)
    mfi = MFI(df['High'], df['Low'], df['Adj Close'],df['Volume'],timeperiod=fast)
    slowk, slowd = STOCH(df['High'], df['Low'], df['Adj Close'], fastk_period=fast, 
                         slowk_period=slow, slowk_matype=0, slowd_period=slow, slowd_matype=0)
    
    #
    #
    #
    
    df['ema'] = E_M_A
    df['rsi'] = R_S_I
    df['macd'] = macd
    df['macdsig'] = macdsig
    df['macdhist'] = macdhist
    df['mfi'] = mfi
    df['slowK'] = slowk
    df['slowD'] = slowd
    
    
    
    ####
    
    

    #76 vars

    #are_all_zero = (test_TA == 0).all()
    #true if all values are 0
    #false if contain a non 0'''

    df.drop(['Close'], axis =1, inplace = True)

    

In [ ]:
def add_lag(num_lag_cols, this_df):
    
    tic = time.perf_counter()
    
    '''df['ema'] = E_M_A
    df['rsi'] = R_S_I
    df['macd'] = macd
    df['macdsig'] = macdsig
    df['macdhist'] = macdhist
    df['mfi'] = mfi
    df['slowK'] = slowk
    df['slowD'] = slowd'''
    
    lag_cols = ['open',
                 'high',
                 'low',
                 'adj_close',
                 'volume',
                 'ema',
                 'rsi',
                 'macd',
                 'macdsig',
                 'macdhist',
                 'mfi',
                 'slowk',
                 'slowd']

    shift_range = [x+1 for x in range(num_lag_cols)]

    for shift in shift_range:
        train_shift = this_df[merging_keys + lag_cols].copy()

        # E.g. order_day of 0 becomes 1, for shift = 1.
        # So when this is merged with order_day of 1 in df, this will represent lag of 1.
        train_shift['order_day'] = train_shift['order_day'] + shift

        foo = lambda x: '{}_lag_{}'.format(x, shift) if x in lag_cols else x
        train_shift = train_shift.rename(columns=foo)

        this_df = pd.merge(this_df, train_shift, on=merging_keys, how='left') #.fillna(0)

    del train_shift
    
    toc = time.perf_counter()

    tic_toc = (toc - tic) / 60

    print(f"completed lagging in {tic_toc:0.4f} min")
    

    return this_df

In [ ]:
def add_scale(num_interval_lag):

    cols_to_scale = ['open',#
                     'high',#
                     'low',#
                     'adj_close',#
                     'volume',#
                     'ema',#
                     'rsi',#
                     'macd',#
                     'macdsig',#
                     'macdhist',#
                     'mfi',#
                     'slowk',#
                     'slowd']#
                    

    for i in range(1,num_interval_lag+1):
        cols_to_scale.append("ema_lag_"+str(i))
        cols_to_scale.append("slowk_lag_"+str(i))
        cols_to_scale.append("slowd_lag_"+str(i))
        cols_to_scale.append("rsi_lag_"+str(i))

        cols_to_scale.append("volume_lag_"+str(i))

        cols_to_scale.append("adj_close_lag_"+str(i))
        
        cols_to_scale.append("open_lag_"+str(i))
        cols_to_scale.append("high_lag_"+str(i))
        cols_to_scale.append("low_lag_"+str(i))

        cols_to_scale.append("macd_lag_"+str(i))
        cols_to_scale.append("macdsig_lag_"+str(i))
        cols_to_scale.append("macdhist_lag_"+str(i))

        cols_to_scale.append("mfi_lag_"+str(i))
      
    return cols_to_scale

In [ ]:
stock = 'MSFT'

start_date = datetime.datetime.now() - datetime.timedelta(days=59)

end_date = datetime.datetime.now()

df = pdr.get_data_yahoo(stock, start=start_date, end=end_date, interval = "2m", prepost = True)

#df = pdr.get_data_yahoo(stock, period = "max", interval = "1d", prepost = True)

#df.index = df.index.tz_localize(None)

'''#2 min ticker
# 30 intervals = 1 hour << OLD
# 195 intervals = trading day'''
                                            # there are more intervals that we can use / change
really_fast = 30
fast = 60
slow = 90

add_indicators()

# Convert Date column to datetime
df.reset_index(level=0, inplace=True)

# Change all column headings to be lower case, and remove spacing
df.columns = [str(x).lower().replace(' ', '_') for x in df.columns]

df['order_day'] = [x for x in list(range(len(df)))]
# merging_keys


# Get difference between high and low of each day

In [ ]:
merging_keys = ['order_day']

num_interval_lag = 30

df = add_lag(num_interval_lag, df)

#df['adj_close'] = df['adj_close'].shift(30)

In [ ]:
window_size = 30

df['adj_close'] = df['adj_close'].shift(-window_size)



In [ ]:
cols_to_rem = ['open',
                 'high',
                 'low',
                 'volume',
                 'ema',
                 'rsi',
                 'macd',
                 'macdsig',
                 'macdhist',
                 'mfi',
                 'slowk',
                 'slowd',
                 'order_day']

In [ ]:
close_vals_list = df['adj_close']

In [ ]:
df.drop(cols_to_rem, axis = 1, inplace = True)

In [ ]:


#close_vals_list = df['adj_close']

df.fillna(0, inplace=True)

#df.fillna(0, inplace=True)

#BUY => 1, SELL => 0, HOLD => 2

num_test = int(.10*len(df))
num_train = len(df) - num_test

# Split into train, cv, and test
train = df[:num_train]
test = df[num_train:]

#print(test['datetime'].iloc[[-1]])
#print(test['test_labels'].iloc[[-1]])

train_close_vals = close_vals_list[:num_train]
test_close_vals = close_vals_list[num_train:]

#test_dates_list = test['date']
#BUY => 1, SELL => 0, HOLD => 2

In [ ]:
test_dates_list = test['datetime']

In [ ]:
train_dates_list = train['datetime']

In [ ]:
cols_to_scale = add_scale(num_interval_lag)
#cols_to_scale.remove(cols_to_rem)

# Do scaling for train set
# Here we only scale the train dataset, and not the entire dataset to prevent information leak


In [ ]:
for element in cols_to_rem:

    if element in cols_to_scale:

        cols_to_scale.remove(element)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import QuantileTransformer

In [ ]:
#scaler = MinMaxScaler(feature_range=(-1,1))

scaler = QuantileTransformer()

scaler.fit(train[cols_to_scale])
train_scaled = scaler.transform(train[cols_to_scale])

# Convert the numpy array back into pandas dataframe

train_scaled = pd.DataFrame(train_scaled, columns=cols_to_scale)

train_scaled = train_scaled[slow:]
train = train[slow:]

In [ ]:
#scaling test dataset


#test = np.array(test, dtype=np.longdouble)
scaler.fit(test[cols_to_scale])
test_scaled = scaler.transform(test[cols_to_scale])

# Convert the numpy array back into pandas dataframe

test_scaled = pd.DataFrame(test_scaled, columns=cols_to_scale)

features = cols_to_scale

In [ ]:
scaled_y_test = test_scaled['adj_close']

In [ ]:
scaled_y_test

In [ ]:
features.remove('adj_close')

In [ ]:
print('adj_close' in features)

In [ ]:
print('adj_close' in train_scaled.columns)

In [ ]:
#features.remove(target)

# Split into X and y
'''X_train_scaled = train_scaled[features]
y_train_scaled = train['labels']

X_test_scaled = test_scaled[features]
y_test_scaled = test['labels']'''

X_train = train_scaled[features]
y_train = train_scaled['adj_close']

X_test = test_scaled[features]
y_test = test_scaled['adj_close']

In [ ]:
import pyswarms as ps

from sklearn.metrics import mean_squared_error

In [ ]:
def fitness_function(position):
    
    num_part, dummy = position.shape
    
    #print(position)
    
    costs = []
    
    for i in range(num_part):
        
    #print(position[i,0])
    #print(position[i,1])


        this_gamma = position[i,0]
        this_sigma = position[i,1]

        #breakpoint()


        clfrbf=LSSVMRegression(
        gamma=this_gamma,       #set the gamma-hyper parameter equal to 1
        kernel='rbf', #use the linear kernel
        sigma=this_sigma
            )

        ##print(clfrbf.get_params())

        clfrbf.fit(X_train, y_train)

        #y_train_pred = clfrbf.predict(X_train)

        y_test_pred = clfrbf.predict(X_test)

        mse_test = mean_squared_error(y_test, y_test_pred)
        
        costs.append(mse_test)

        #rmse_test = math.sqrt(mse_test)

        #print('results for round: cost = {cost}, sigma = {sigma}, Gamma={gamma}'
             # .format(cost = mse_test, sigma=this_sigma, gamma=this_gamma))

    #mse_train = mean_squared_error(y_train, y_train_pred)

    #rmse_train = math.sqrt(mse_train)

    #list(range(mse_f_train,rmse_f_train))
    
    
    #print(costs)

    
    return costs
    #Return a vector instead of single value. you can use the rmse_test and ignore rmse_train
    
    #svrRegressor = SVR(kernel='rbf', gamma = position[0].all(), C = position[1].all() )
    


In [ ]:
# I wonder if reducing the # of features / lagging features increases training speed?
# theoretical to
# investigate numba library to do GPU calculations
#parralelization?

In [ ]:
lb = [0.01, 0.01]
ub = [1000,1000]
#mb = [100, 0.001]
bounds = (lb,ub)

options = {'c1': 0.5, 'c2': 0.3, 'w':0.9}
optimizer = ps.single.GlobalBestPSO(n_particles=12, dimensions=2,options=options, bounds = bounds, ftol = .5)
best_cost, opt_params = optimizer.optimize(fitness_function, iters=5, n_processes=12)

In [ ]:
print(best_cost)
print(opt_params)

In [ ]:
clfrbf=LSSVMRegression(
        gamma=842.873436731312,       #set the gamma-hyper parameter equal to 1
        kernel='rbf', #use the linear kernel
        sigma=310.9203341185021
            )

In [ ]:
clfrbf.fit(X_train, y_train)

In [ ]:
clfrbf.get_params()

In [ ]:
y_hat = clfrbf.predict(X_test)

In [ ]:
y_hat

In [ ]:
y_hat_train = clfrbf.predict(X_train)

In [ ]:
y_hat_train

In [ ]:
df_to_descale = test_scaled.copy()

In [ ]:
df_to_descale['adj_close'] = y_hat

In [ ]:


inv_scaler = scaler.inverse_transform(df_to_descale)

y_hat_descaled = pd.DataFrame(inv_scaler, columns = test_scaled.columns)

In [ ]:
#y_hat_descaled['adj_close'].head(30)

In [ ]:
y_hat

In [ ]:
mse_test =mean_squared_error(y_test, y_hat)
print(mse_test)

In [ ]:
%matplotlib notebook
plt.rcParams['figure.figsize'] = [10, 6]

plt.plot(test_dates_list, y_hat_descaled['adj_close'])
plt.plot(test_dates_list, test_close_vals)
plt.legend(['pred','true'])
plt.show

In [ ]:
train_scaled['adj_close'] = y_hat_train

In [ ]:
train_inv_scaler = scaler.inverse_transform(train_scaled)

y_hat_train_descaled = pd.DataFrame(train_inv_scaler, columns = train_scaled.columns)

In [ ]:
%matplotlib notebook
plt.rcParams['figure.figsize'] = [10, 6]

plt.plot(train_dates_list[slow:], y_hat_train_descaled['adj_close'])
plt.plot(train_dates_list, train_close_vals)
plt.legend(['pred','true'])
plt.show